In [144]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [145]:
data=pd.read_csv("/kaggle/input/titanic/train.csv")
test=pd.read_csv("/kaggle/input/titanic/test.csv")
psg_id=test["PassengerId"]
# Drop columns
data.drop(["Name", "Ticket", "PassengerId", "Cabin"], axis=1, inplace=True)
test.drop(["Name", "Ticket", "PassengerId", "Cabin"], axis=1, inplace=True)

# Fill NaN values
cols = ["SibSp", "Parch", "Fare", "Age"]
for col in cols:
    data[col] = data[col].fillna(data[col].median())
    test[col] = test[col].fillna(test[col].median())

data["Embarked"] = data["Embarked"].fillna("U")
test["Embarked"] = test["Embarked"].fillna("U")


In [146]:
data.head(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [147]:
from sklearn import preprocessing
le=preprocessing.LabelEncoder()

cols=["Sex","Embarked"]
for col in cols:
    data[col]=le.fit_transform(data[col])
    test[col]=le.transform(test[col])
    print(le.classes_)
    
data.head(10)

['female' 'male']
['C' 'Q' 'S' 'U']


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2
3,1,1,0,35.0,1,0,53.1000,2
4,0,3,1,35.0,0,0,8.0500,2
5,0,3,1,28.0,0,0,8.4583,1
6,0,1,1,54.0,0,0,51.8625,2
7,0,3,1,2.0,3,1,21.0750,2
8,1,3,0,27.0,0,2,11.1333,2
9,1,2,0,14.0,1,0,30.0708,0


In [148]:
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split;

y=data["Survived"]
x=data.drop("Survived",axis=1)

x_train,x_val,y_train,y_val=train_test_split(x,y,test_size=0.2,random_state=42)

In [149]:
 clf=LogisticRegression(random_state=0,max_iter=1500).fit(x_train,y_train)
#clf=LinearRegression().fit(x_train,y_train)



In [150]:
predictions=clf.predict(x_val)

#from sklearn.preprocessing import Binarizer
# Create a Binarizer instance with threshold 0.5
#binarizer = Binarizer(threshold=0.5)
# Transform y_val to binary values (0 or 1)
#predictions = binarizer.transform(predictions.reshape(-1, 1))

from sklearn.metrics import accuracy_score
accuracy_score(y_val,predictions)

0.8100558659217877

In [151]:
submission_preds=clf.predict(test)
df=pd.DataFrame({"PassengerId":psg_id,"Survived":submission_preds,})
df.to_csv("submission.csv",index=False)